In [2]:
import numpy as np
import cv2
import imutils
#import sys
#import cv2.aruco as aruco


In [3]:
#Load the predefined dictionary
dict_aruco= cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_250)

# generate the markers

for i in range(4):
    marker_image= np.zeros((200,200), dtype=np.uint8)
    marker_image= cv2.aruco.drawMarker(dict_aruco,i+2, 200, marker_image,2)
    filename= "AURCO_" + str(i) +".png"
    cv2.imwrite(filename,marker_image)
marker_image= cv2.aruco.drawMarker(dict_aruco,4, 200, marker_image,2)
cv2.imwrite("AURCO_2.png",marker_image)


True

In [4]:
def findArucoMarkers(img, markerSize = 6, totalMarkers=250, draw=True):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    key = getattr(cv2.aruco, f'DICT_{markerSize}X{markerSize}_{totalMarkers}')
    
    
    #Load the dictionary that was used to generate the markers.
    arucoDict = cv2.aruco.Dictionary_get(key)
    
    # Initialize the detector parameters using default values
    arucoParam = cv2.aruco.DetectorParameters_create()
    
    # Detect the markers
    bboxs, ids, rejected = cv2.aruco.detectMarkers(gray, arucoDict, parameters = arucoParam)
    return bboxs, ids

In [5]:
imgH=480
imgW=640

video = cv2. VideoCapture(0)

ret, video_frame=video.read()
image = cv2.imread(r'nature.png')
image = cv2.resize(image, (imgH, imgW))
#imgH, imgW= image.shape[:2]
cv2.imshow("Image Renu",image)
while(video.isOpened()):
    if ret==True:
        refPts=[]
        #source = imutils.resize(source , width=imgH, height=imgH)
        #cv2.imshow("Source",source)
        arucofound =findArucoMarkers(video_frame, totalMarkers=100)

        h, w = video_frame.shape[:2]
        print("height",h,w)
        #print(len(arucofound), "aruco", arucofound)
        if  len(arucofound[0])!=0:
                #refPts=[]
                for Corner, id in zip(arucofound[0], arucofound[1]):
                    
                    print("ID",id,"corner",Corner)
                    corners = Corner.reshape((4, 2))
                    (topLeft, topRight, bottomRight, bottomLeft) = corners
                    # convert each of the (x, y)-coordinate pairs to integers
                    topRight = (int(topRight[0]), int(topRight[1]))
                    bottomRight = (int(bottomRight[0]), int(bottomRight[1]))
                    bottomLeft = (int(bottomLeft[0]), int(bottomLeft[1]))
                    topLeft = (int(topLeft[0]), int(topLeft[1]))
                    #cv2.imshow('img',image)
                    # draw the bounding box of the ArUCo detection
                    cv2.line(video_frame, topLeft, topRight, (0, 255, 0), 2)
                    cv2.line(video_frame, topRight, bottomRight, (0, 255, 0), 2)
                    cv2.line(video_frame, bottomRight, bottomLeft, (0, 255, 0), 2)
                    cv2.line(video_frame, bottomLeft, topLeft, (0, 255, 0), 2)
                    
                    cv2.putText(video_frame, str(id),(topLeft[0], topLeft[1] - 15), cv2.FONT_HERSHEY_SIMPLEX,0.5, (0, 255, 0), 2)
                    corner = np.squeeze(Corner)
                    refPts.append(corner)

                    print("refpoints",refPts)
                    print("length",len(refPts))
                    if len(refPts)==4:
                        ( refPtBR, refPtTR,refPtBL, refPtTL) = refPts
                        #correct
                        #source_pt = np.array([   refPtBL[3],refPtBR[3],refPtTL[3], refPtTR[3],])
                        video_pt = np.array([  refPtTL[3], refPtBL[3],refPtBR[3], refPtTR[2]])
                        #source_pt = np.array( [refPtTR[3],refPtTL[3], refPtBR[2], refPtBL[3]])
                        
                        print("source_pt", video_pt)
                        # grab the spatial dimensions of the source image and define the
                        # transform matrix for the *source* image in top-left, top-right,
                        # bottom-right, and bottom-left order
                        image_pt = np.float32([[0,0], [h,0], [h,w], [0,w]])
                        print("dest",image_pt)
                        # compute the homography matrix and then warp the source image to the
                        # destination based on the homography
                        matrix, _ = cv2.findHomography( image_pt, video_pt)
                        warped  = cv2.warpPerspective(image, matrix, (video_frame.shape[1], video_frame.shape[0]))
                        print(warped.shape)
                        cv2.imshow("warp inside", warped)
                        mask = np.zeros((imgH, imgW), dtype="uint8")
                        cv2.fillConvexPoly(mask, video_pt.astype("int32"), (255, 255, 255),cv2.LINE_AA)
                        #imgout = image + warped
                        # this step is optional, but to give the source image a black border
                        # surrounding it when applied to the source image, you can apply a
                        # dilation operation
                        rect = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
                        mask = cv2.dilate(mask, rect, iterations=2)
                        # create a three channel version of the mask by stacking it depth-wise,
                        # such that we can copy the warped source image into the input image
                        maskScaled = mask.copy() / 255.0
                        maskScaled = np.dstack([maskScaled] * 3)
                        print(maskScaled.shape)
                        print("Image",video_frame.shape)

                        # copy the warped source image into the input image by (1) multiplying
                        # the warped image and masked together, (2) multiplying the original
                        # input image with the mask (giving more weight to the input where
                        # there *ARE NOT* masked pixels), and (3) adding the resulting
                        # multiplications together
                        warpedMultiplied = cv2.multiply(warped.astype("float"), maskScaled)
                        imageMultiplied = cv2.multiply(video_frame.astype(float), 1.0 - maskScaled)
                        output = cv2.add(warpedMultiplied, imageMultiplied)
                        output = output.astype("uint8")
                        cv2.imshow("output", output)
    cv2.imshow("Source",video_frame)
    ret, video_frame=video.read()
    key = cv2.waitKey(20)
    if key == 113:
        break 
    
# and finally destroy/close all open windows
video.release()
cv2.destroyAllWindows()

height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
ID [62] corner [[[366. 336.]
  [363. 317.]
  [381. 314.]
  [384. 333.]]]
refpoints [array([[366., 336.],
       [363., 317.],
       [381., 314.],
       [384., 333.]], dtype=float32)]
length 1
height 480 640
ID [62] corner [[[366. 336.]
  [363. 317.]
  [381. 314.]
  [384. 333.]]]
refpoints [array([[366., 336.],
       [363., 317.],
       [381., 314.],
       [384., 333.]], dtype=float32)]
length 1
height 480 640
ID [98] corner [[[268. 335.]
  [270. 354.]
  [250. 356.]
  [248. 338.]]]
refpoints [array([[268., 335.],
       [270., 354.],
       [250., 356.],
       [248., 338.]], dtype=float32)]
length 1
ID [62] corner [[[365. 337.]
  [362. 317.]
  [380. 315.]
  [383. 334.]]]
refpoints [array([[268., 335.],
       [270., 354.],
       [250., 356.],
       [248., 338.]], dtype=float32), array([[365., 337.],
       [362., 317.],
       [380., 315.],
       [383., 334.]], dtype=float32

refpoints [array([[265., 316.],
       [267., 335.],
       [248., 337.],
       [246., 318.]], dtype=float32)]
length 1
ID [62] corner [[[363. 318.]
  [360. 299.]
  [378. 296.]
  [382. 315.]]]
refpoints [array([[265., 316.],
       [267., 335.],
       [248., 337.],
       [246., 318.]], dtype=float32), array([[363., 318.],
       [360., 299.],
       [378., 296.],
       [382., 315.]], dtype=float32)]
length 2
height 480 640
ID [98] corner [[[265. 315.]
  [267. 334.]
  [248. 336.]
  [246. 317.]]]
refpoints [array([[265., 315.],
       [267., 334.],
       [248., 336.],
       [246., 317.]], dtype=float32)]
length 1
ID [62] corner [[[362. 316.]
  [360. 297.]
  [379. 295.]
  [382. 314.]]]
refpoints [array([[265., 315.],
       [267., 334.],
       [248., 336.],
       [246., 317.]], dtype=float32), array([[362., 316.],
       [360., 297.],
       [379., 295.],
       [382., 314.]], dtype=float32)]
length 2
ID [40] corner [[[259. 142.]
  [258. 158.]
  [240. 156.]
  [241. 140.]]]
refpoin

height 480 640
ID [62] corner [[[363. 289.]
  [359. 269.]
  [379. 267.]
  [382. 285.]]]
refpoints [array([[363., 289.],
       [359., 269.],
       [379., 267.],
       [382., 285.]], dtype=float32)]
length 1
ID [98] corner [[[258. 284.]
  [260. 304.]
  [240. 306.]
  [238. 286.]]]
refpoints [array([[363., 289.],
       [359., 269.],
       [379., 267.],
       [382., 285.]], dtype=float32), array([[258., 284.],
       [260., 304.],
       [240., 306.],
       [238., 286.]], dtype=float32)]
length 2
ID [23] corner [[[362. 123.]
  [344. 127.]
  [343. 111.]
  [361. 108.]]]
refpoints [array([[363., 289.],
       [359., 269.],
       [379., 267.],
       [382., 285.]], dtype=float32), array([[258., 284.],
       [260., 304.],
       [240., 306.],
       [238., 286.]], dtype=float32), array([[362., 123.],
       [344., 127.],
       [343., 111.],
       [361., 108.]], dtype=float32)]
length 3
height 480 640
ID [98] corner [[[259. 281.]
  [261. 301.]
  [240. 303.]
  [239. 283.]]]
refpoints [a

height 480 640
ID [62] corner [[[390. 258.]
  [386. 237.]
  [408. 235.]
  [413. 255.]]]
refpoints [array([[390., 258.],
       [386., 237.],
       [408., 235.],
       [413., 255.]], dtype=float32)]
length 1
ID [23] corner [[[381.  83.]
  [362.  87.]
  [359.  70.]
  [379.  67.]]]
refpoints [array([[390., 258.],
       [386., 237.],
       [408., 235.],
       [413., 255.]], dtype=float32), array([[381.,  83.],
       [362.,  87.],
       [359.,  70.],
       [379.,  67.]], dtype=float32)]
length 2
height 480 640
ID [62] corner [[[390. 258.]
  [386. 237.]
  [408. 235.]
  [413. 255.]]]
refpoints [array([[390., 258.],
       [386., 237.],
       [408., 235.],
       [413., 255.]], dtype=float32)]
length 1
ID [23] corner [[[381.  83.]
  [362.  87.]
  [359.  70.]
  [379.  67.]]]
refpoints [array([[390., 258.],
       [386., 237.],
       [408., 235.],
       [413., 255.]], dtype=float32), array([[381.,  83.],
       [362.,  87.],
       [359.,  70.],
       [379.,  67.]], dtype=float32)]
l

height 480 640
ID [98] corner [[[300. 268.]
  [303. 291.]
  [280. 293.]
  [278. 270.]]]
refpoints [array([[300., 268.],
       [303., 291.],
       [280., 293.],
       [278., 270.]], dtype=float32)]
length 1
ID [62] corner [[[414. 274.]
  [409. 253.]
  [431. 251.]
  [434. 272.]]]
refpoints [array([[300., 268.],
       [303., 291.],
       [280., 293.],
       [278., 270.]], dtype=float32), array([[414., 274.],
       [409., 253.],
       [431., 251.],
       [434., 272.]], dtype=float32)]
length 2
ID [23] corner [[[406.  94.]
  [386.  97.]
  [384.  80.]
  [404.  78.]]]
refpoints [array([[300., 268.],
       [303., 291.],
       [280., 293.],
       [278., 270.]], dtype=float32), array([[414., 274.],
       [409., 253.],
       [431., 251.],
       [434., 272.]], dtype=float32), array([[406.,  94.],
       [386.,  97.],
       [384.,  80.],
       [404.,  78.]], dtype=float32)]
length 3
height 480 640
ID [98] corner [[[302. 267.]
  [305. 289.]
  [282. 292.]
  [279. 270.]]]
refpoints [a

...

height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
ID [62] corner [[[460.  93.]
  [455.  77.]
  [474.  72.]
  [479.  87.]]]
refpoints [array([[460.,  93.],
       [455.,  77.],
       [474.,  72.],
       [479.,  87.]], dtype=float32)]
length 1
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 

height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480

height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480

height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480

height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480 640
height 480

In [6]:
imgH=480
imgW=640
fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
video_write=cv2.VideoWriter('videoAR.avi',fourcc,1,(imgW,imgH))
video = cv2. VideoCapture(0)

ret, video_frame=video.read()
image = cv2.imread(r'nature.png')
image = cv2.resize(image, (imgH, imgW))

while(video.isOpened()):
    if ret==True:
        cv2.imshow("Source", video_frame)
        refPts=[]  
        #Detect the Aruco markers on the video frame
        arucofound =findArucoMarkers(video_frame, totalMarkers=100)
        h, w = video_frame.shape[:2]
       
        if  len(arucofound[0])!=0:
                
                for Corner, id in zip(arucofound[0], arucofound[1]):
                    
                    corners = Corner.reshape((4, 2))
                    (topLeft, topRight, bottomRight, bottomLeft) = corners
                    topRight = (int(topRight[0]), int(topRight[1]))
                    bottomRight = (int(bottomRight[0]), int(bottomRight[1]))
                    bottomLeft = (int(bottomLeft[0]), int(bottomLeft[1]))
                    topLeft = (int(topLeft[0]), int(topLeft[1]))
                    cv2.line(video_frame, topLeft, topRight, (0, 255, 0), 2)
                    cv2.line(video_frame, topRight, bottomRight, (0, 255, 0), 2)
                    cv2.line(video_frame, bottomRight, bottomLeft, (0, 255, 0), 2)
                    cv2.line(video_frame, bottomLeft, topLeft, (0, 255, 0), 2)                    
                    cv2.putText(video_frame, str(id),(topLeft[0], topLeft[1] - 15), cv2.FONT_HERSHEY_SIMPLEX,0.5, (0, 255, 0), 2)
                    corner = np.squeeze(Corner)
                    refPts.append(corner)
                    if len(refPts)==4:
                        ( refPtBR, refPtTR,refPtBL, refPtTL) = refPts
                        # , top left,top right , bottom left, bottom right
                        video_pt = np.array([  refPtTL[3], refPtBL[3],refPtBR[2], refPtTR[3]])
                       
                        # grab the spatial dimensions of the  image and define the
                        # transform matrix for the image in 
                        #top-left, top-right,bottom-right, and bottom-left order
                        image_pt = np.float32([[0,0], [h,0], [h,w], [0,w]])
                        
                        # compute the homography matrix between the image and the video frame
                        matrix, _ = cv2.findHomography( image_pt, video_pt)
                        
                        #warp the  image to video frame based on the homography
                        warped  = cv2.warpPerspective(image, matrix, (video_frame.shape[1], video_frame.shape[0]))
                        
                        #Create a mask representing region to 
                        #copy from the warped image into the video frame.
                        mask = np.zeros((imgH, imgW), dtype="uint8")
                        cv2.fillConvexPoly(mask, video_pt.astype("int32"), (255, 255, 255),cv2.LINE_AA)
                                                                    
                        # give the source image a black border
                        # surrounding it when applied to the source image,
                        #you can apply a dilation operation
                        rect = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
                        mask = cv2.dilate(mask, rect, iterations=2)
                        
                        # Copy the mask with the three channel version by stacking it depth-wise,
                        # This will allow copying the warped source image into the input image
                        maskScaled = mask.copy() / 255.0
                        maskScaled = np.dstack([maskScaled] * 3)
                        
                        # Copy the masked warped image into the video frame by
                        # (1) multiplying the warped image and masked together, 
                        # (2) multiplying the Video frame with the mask 
                        # (3) adding the resulting images
                        warpedMultiplied = cv2.multiply(warped.astype("float"), maskScaled)
                        imageMultiplied = cv2.multiply(video_frame.astype(float), 1.0 - maskScaled)
                        #imgout = video frame multipled with mask 
                        #        + warped image multipled with mask
                        output = cv2.add(warpedMultiplied, imageMultiplied)
                        output = output.astype("uint8")
                        cv2.imshow("Source with id", video_frame)
                        cv2.imshow("output", output)
                        video_write.write(output)
    ret, video_frame=video.read()
    key = cv2.waitKey(20)
    # if key q is pressed then break 
    if key == 113:
        break 
    
#finally destroy/close all open windows
video.release()
cv2.destroyAllWindows()